# Gallery Example: M/M/1 Tandem Multiclass Network

This example demonstrates a 2-queue tandem network with multiple classes:
- **Arrivals**: Exponential (separate Poisson process per class)
- **Service**: Exponential (class-dependent rates)
- **Servers**: 1 server per queue
- **Scheduling**: FCFS
- **Classes**: 2 job classes with different arrival and service rates

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_tandem_multiclass(): Network {
    """Create M[2]/M[2]/1 tandem multiclass queueing network"""
    val model = Network("M[2]/M[2]/1 -> -/M[2]/1")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
    val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass1 = OpenClass(model, "myClass1")
    source.setArrival(oclass1, Exp(1))
    queue1.setService(oclass1, Exp(4))
    queue2.setService(oclass1, Exp(6))
    
    val oclass2 = OpenClass(model, "myClass2")
    source.setArrival(oclass2, Exp(0.5))
    queue1.setService(oclass2, Exp(2))
    queue2.setService(oclass2, Exp(6))
// Block 3: topology
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue1, 1.0)
    P.addRoute(oclass1, queue1, queue2, 1.0)
    P.addRoute(oclass1, queue2, sink, 1.0)
    P.addRoute(oclass2, source, queue1, 1.0)
    P.addRoute(oclass2, queue1, queue2, 1.0)
    P.addRoute(oclass2, queue2, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_mm1_tandem_multiclass()

## About Multiclass Tandem Networks

This model has two classes with different characteristics:

**Class 1**:
- Arrival rate: λ₁ = 1
- Service rate at Queue1: μ₁₁ = 4
- Service rate at Queue2: μ₁₂ = 6

**Class 2**:
- Arrival rate: λ₂ = 0.5
- Service rate at Queue1: μ₂₁ = 2
- Service rate at Queue2: μ₂₂ = 6

With FCFS scheduling, both classes share the server capacity but have different service requirements.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)